In [1]:
# Initial setup by importing needed modules
import datetime as dt

import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [22]:
# Read off data and put that into a panda dataframe
lend_3a = pd.read_csv('data/LoanStats3a.csv', header=1, low_memory=False)
lend_3b = pd.read_csv('data/LoanStats3b.csv', header=1, low_memory=False)
lend = lend_3a.append(lend_3b)

#### Understanding data structure, checking shape and data type

In [23]:
print(lend.shape)
lend.info()

(230721, 144)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 230721 entries, 0 to 188182
Columns: 144 entries, id to settlement_term
dtypes: float64(108), object(36)
memory usage: 255.2+ MB


There are 144 columns, 108 of those are floats. Let's look at floats columns first. 

In [24]:
lend.select_dtypes(include='float').head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,NaN,5000.0,5000.0,4975.0,162.87,24000.0,NaN,27.65,0.0,1.0,NaN,NaN,3.0,0.0,13648.0,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,171.62,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2500.0,2500.0,2500.0,59.83,30000.0,NaN,1.00,0.0,5.0,NaN,NaN,3.0,0.0,1687.0,4.0,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,119.66,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2400.0,2400.0,2400.0,84.33,12252.0,NaN,8.72,0.0,2.0,NaN,NaN,2.0,0.0,2956.0,10.0,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,649.91,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,10000.0,10000.0,10000.0,339.31,49200.0,NaN,20.00,0.0,1.0,35.0,NaN,10.0,0.0,5598.0,37.0,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,357.48,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,3000.0,3000.0,3000.0,67.79,80000.0,NaN,17.94,0.0,0.0,38.0,NaN,15.0,0.0,27783.0,38.0,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,67.30,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N

While they are relevant information on describing the loans. We observed two issues:
1. The problem is to predict the loan default/paid off chances at the point of application. Details about the loans (such as terms, interest rates, hardship_length, etc.) are unknown upfront. Those columns cannot be used in the machine learning analysis.
2. Some columns appears to be all missing values. This bring us concerns on whether the dataset is complete. We will look into that soon.

Fortunately, the two types of information do not appear to be relevant to the machine learning problems as they cannot be known in advance or generic information (e.g. members id) that do not have predicting power.

Now we will review the 36 object columns. 

In [25]:
lend.select_dtypes(exclude='float').head()

,id,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,NaN,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,n,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,NaN,May-2019,Individual,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
1,NaN,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,n,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,NaN,Oct-2016,Individual,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
2,NaN,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,n,NaN,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,NaN,Jun-2017,Individual,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,NaN,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,n,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,Feb-1996,21%,f,Jan-2015,NaN,Apr-2016,Individual,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,NaN,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Fully Paid,n,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,Jan-1996,53.9%,f,Jan-2017,NaN,Apr-2018,Individual,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN


Some of the columns actually have date time or percentage values, but panda cannot recognize them in the initial load. Those columns will need data transformation.

#### Handling missing values

In [27]:
# Look at how many missing records by columns
lend.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,230716.000000,230716.000000,230716.000000,230716.000000,2.307120e+05,0.0,230716.000000,230687.000000,230687.000000,96217.000000,21125.000000,230687.000000,230687.000000,2.307160e+05,230687.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230716.000000,230571.000000,32516.000000,230716.0,0.0,0.0,0.0,230687.000000,160440.000000,1.604400e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.604400e+05,0.0,0.0,0.0,180686.00000,160434.000000,179156.000000,179069.000000,230571.000000,230687.000000,154309.000000,160439.000000,160439.000000,160440.000000,180686.000000,179353.00000,36751.000000,160313.00000,54447.000000,160440.000000,160440.000000,160440.000000,172126.000000,160440.000000,160440.000000,160440.000000,160440.000000,160440.000000,172126.000000,160184.000000,160440.000000,160440.000000,160440.000000,160287.000000,179153.000000,229351.000000,230611.000000,1.604400e+05,1.806860e+05,180686.000000,1.604400e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,80.000000,80.0,80.000000,69.000000,80.000000,80.000000,2222.000000,2222.000000,2222.000000
mean,NaN,13752.308683,13700.819731,13565.362454,421.405073,7.166241e+04,NaN,16.380409,0.223563,0.854855,34.984265,81.374533,10.695518,0.097357,1.594589e+04,24.097725,0.001405,0.001405,15527.430763,15386.372455,12223.559444,3123.663283,1.114684,179.093359,21.375038,3883.375248,0.002589,41.778294,1.0,NaN,NaN,NaN,0.002232,76.736188,1.373241e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.989160e+04,NaN,NaN,NaN,3.92953,13796.105776,8262.724893,66.830442,0.004298,6.851634,125.076787,178.488491,14.108459,8.938581,1.811020,25.65990,40.818399,6.99123,36.612118,0.329506,3.755192,5.675374,4.665820,9.018979,7.726427,8.094459,14.950405,5.693393,11.085368,0.000474,0.002013,0.063737,1.788014,95.399571,53.558127,0.077684,0.011461,1.655453e+05,4.288377e+04,20238.462072,3.438956e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,96.949250,3.0,13.525000,291.816957,6291.665375,170.367250,4190.399158,46.130729,4.881638
std,NaN,8089.315652,8060.474691,8101.558643,241.405132,5.430890e+04,NaN,7.580711,0.673428,1.145356,21.745285,32.634309,4.632031,0.382305,1.

The dataset has a total of 200K row of data. Some columns almost have no data at all (count = 0), and wouldn't be useful. By scanning the missing records range, I will make the decision to drop columns that have >100K missing records. Any Loan_amnt records which are missing will be dropped as well.

In [30]:
lend_drop = lend.dropna(axis=1, how='all', thresh=100000).dropna(
    subset=['loan_amnt'])
lend_drop.shape

(230716, 87)

#### Why some columns have almost no data?

By looking at the columns, I come up with two reasons:
1. Sensitive information - columns such as member id are personal privacy. Lending club removed that information on purpose.
2. Information not applicable to closed loans. Column such as settlement amount may only apply to active loans which are still in progress on being repaid and not for loans that have been paid off (or default).

Fortunately, the two types of information do not appear to be relevant to the machine learning problems as they cannot be known in advance or generic information (e.g. members id) that do not have predicting power.

For the float type columns, let's have another look into what those columns are

In [31]:
lend_drop.select_dtypes(include='float').isnull().sum()

loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
installment                       0
annual_inc                        4
dti                               0
delinq_2yrs                      29
inq_last_6mths                   29
open_acc                         29
pub_rec                          29
revol_bal                         0
total_acc                        29
out_prncp                         0
out_prncp_inv                     0
total_pymnt                       0
total_pymnt_inv                   0
total_rec_prncp                   0
total_rec_int                     0
total_rec_late_fee                0
recoveries                        0
collection_recovery_fee           0
last_pymnt_amnt                   0
collections_12_mths_ex_med      145
policy_code                       0
acc_now_delinq                   29
tot_coll_amt                  70276
tot_cur_bal                   70276
total_rev_hi_lim            

After reviewing the data dictionary for those columns, I conclude that the missing values would imply zeros. Apply fillna(0) mention to them during the data transformation steps.

#### Data Transformation

For the object columns, further data type transformations are needed. We will focus on issue_d, emp_length, earliest_cr_line, revol_util and title.

#### 1. Loan issues date - convert into datetime value

In [47]:
# use assign function to avoid security type warining
lend_drop = lend_drop.assign(
    loan_start_d=pd.to_datetime(lend_drop.issue_d, format='%b-%Y'))
lend_drop['loan_start_d'].head()

0   2011-12-01
1   2011-12-01
2   2011-12-01
3   2011-12-01
4   2011-12-01
Name: loan_start_d, dtype: datetime64[ns]

#### 2. Employment duration
- convert from text to numeric value. 
- clean up na value, assume <1 year if info is not provided (currently unemployment)

In [34]:
lend_drop['yr_emp'] = lend_drop['emp_length'].str[0:2].replace(
    '< ', 0).astype('float')

#### 3. Credit History
- Convert earliest_cr_line into from text to datetime values
- Replace missing values with zeros
- Extract year in order to obtain categorical values

In [40]:
lend_drop.earliest_cr_line = pd.to_datetime(
    lend_drop.earliest_cr_line, format='%b-%Y')
lend_drop['yr_credit'] = (2014-lend_drop.earliest_cr_line.dt.year).fillna(0)

#### 4. Credit line % usage
- Convert to float
- Replace missing values with zeros

In [41]:
lend_drop['revol_util_dec'] = lend_drop['revol_util'].str.replace(
    r'%', r'0').astype('float')/100

#### 5. Title

In [42]:
# Preview on information in this column
lend_drop.title.head(15)

0                                  Computer
1                                      bike
2                      real estate business
3                                  personel
4                                  Personal
5     My wedding loan I promise to pay back
6                                      Loan
7                           Car Downpayment
8      Expand Business & Buy Debt Portfolio
9               Building my credit history.
10               High intrest Consolidation
11                            Consolidation
12                                  freedom
13                            citicard fund
14                               Other Loan
Name: title, dtype: object

Title describes the purpose of the loan and is useful information. However, in the same dataset, there is another column called **purpose** which resembles information under Title. The purpose column has better precise label. Hence, we won't be using Title column.

In [14]:
lend_drop.purpose.value_counts()

debt_consolidation    49487
credit_card           15502
other                  7054
home_improvement       5878
major_purchase         3548
small_business         3217
car                    2452
wedding                1684
medical                1325
moving                  982
house                   800
vacation                725
educational             325
renewable_energy        174
Name: purpose, dtype: int64

#### Obtain Target variable

loan_status will serve as target for this analysis. Find out what the type of phrases exist in the loan_status column

In [43]:
lend_drop.loan_status.value_counts()

Fully Paid                                             192626
Charged Off                                             35340
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Late (31-120 days)                                          1
Name: loan_status, dtype: int64

Per research through discussion forum, "Dose not meet the credit policy' are loans that had issued before Lending Club update credit policy. For the purpose of this study, we will treat them as "Fully Paid" or "Charged Off', regardless of credit policy compliance. Only one record with label 'late', we will group that into "Charged Off' category for now.

Convert text value feature into numerics for model training purposes. Create a dictionary to convert loan status into 0 (default) or 1 (paid off)

In [45]:
loan_dict = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid': 1,
             'Does not meet the credit policy. Status:Charged Off': 0, 'Late (31-120 days)': 0}

Create target column by mapping loan status to dictionary,

In [46]:
# use assign function to avoid security type warining
lend_drop = lend_drop.assign(target=lend_drop.loan_status.map(loan_dict))

Save cleaned data set into a csv file for quicker access. For machine learning purpose, we need to have hold out set for model testing purposes. We will pick data on 2013 as test set

In [49]:
lend_drop[lend_drop.loan_start_d <
          '2013-1-1'].to_csv('data/lending_clean.csv', index=False)
lend_drop[lend_drop.loan_start_d >=
          '2013-1-1'].to_csv('data/lending_test.csv', index=False)

[Back to README](https://github.com/sittingman/lending_repayment)